In [31]:
!pip install requests
!pip install params
!pip install response


In [62]:
!pip install parse

In [78]:
!pip install predict

In [3]:
!pip install AsciiTable

     |████████████████████████████████| 320 kB 9.0 MB/s 
  Created wheel for AsciiTable: filename=asciitable-0.8.0-py3-none-any.whl size=45593 sha256=25a6e22d53121ec2688d8f9aaa465d7bfaac769eccdef22ad65ca70ae19659cc
  Stored in directory: /root/.cache/pip/wheels/a1/a7/66/9896fd7df73a6c8c06b2b1fe1973fdcb8275828c6a81c4cfdc
Successfully built AsciiTable


In [5]:
!pip install terminaltables 

In [41]:
!pip install -r requirements.txt

In [79]:
!pip install python-dotenv

In [80]:
import requests
from dotenv import load_dotenv
import os
from terminaltables import AsciiTable

In [81]:
def predict_salary(salary_from, salary_to):
    if salary_from > 0 and salary_to > 0:
        mid_salary = int((salary_from + salary_to) / 2)
    elif salary_from > 0:
        mid_salary = int(salary_from * 1.2)
    elif salary_to > 0:
        mid_salary = int(salary_to * 0.8)
    return mid_salary

In [82]:
def predict_rub_salary_hh(vacancy):
    hh_rub_salary = 0
    if vacancy['salary'] and vacancy['salary']['currency'] == 'RUR':
        salary_from = vacancy['salary']['from'] or 0
        salary_to = vacancy['salary']['to'] or 0
        hh_rub_salary = predict_salary(salary_from, salary_to)
    return hh_rub_salary


In [83]:
def predict_rub_salary_sj(vacancy):
    sj_predict_salary = 0
    salary_from = vacancy['payment_from']
    salary_to = vacancy['payment_to']
    if vacancy['currency'] == 'rub':
        sj_predict_salary = predict_salary(salary_from, salary_to)
    return sj_predict_salary

In [84]:
def get_vacancies_from_hh(language):
    hh_url = 'https://api.hh.ru/vacancies'
    page = 0  # первая страница поиска (нумерация с 0)
    number_of_pages = 1
    vacancies = []
    while page < number_of_pages:
        headers = {'User-Agent': 'HH-User-Agent'}
        params = {
            'area': 1,  # Код города, 1 - Москва
            'text': 'программист ' + language,
            'page': page  # Текущая страница поиска
        }
        response = requests.get(hh_url, headers=headers, params=params)
        page += 1
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            continue
        response_data = response.json()
        vacancies.extend(response_data['items'])
        number_of_pages = response_data['pages'] - 1
    return vacancies

In [85]:
def get_hh_statistic(vacancies):
    if vacancies:
        number_of_vacancies = len(vacancies)
        salaries = [predict_rub_salary_hh(vacancy) for vacancy in vacancies if predict_rub_salary_hh(vacancy) != 0]
        vacancies_processed = len(salaries)
        average_salary = int(sum(salaries)/vacancies_processed)
        hh_statistics = {
            'vacancies_found': number_of_vacancies,
            'vacancies_processed': vacancies_processed,
            'average_salary': average_salary
        }
        return hh_statistics

In [86]:
def make_all_language_stat_from_hh(languages):
    stat = {}
    for language in languages:
        vacancies = get_vacancies_from_hh(language)
        stat[language] = get_hh_statistic(vacancies)
    return stat

In [87]:
def get_vacancies_from_sj(language):
    secret_key = os.getenv('SJ_KEY')
    sj_url = 'https://api.superjob.ru/2.0/vacancies'
    page = 0
    next_page = True
    vacancies = []
    while next_page:
        headers = {'X-Api-App-Id': secret_key}
        params = {
            'page': page,  # Номер страницы результата поиска
            'count': 5,  # Количество результатов на страницу поиска
            'keyword': language,  # Язык программирования
            'town': 4,  # Название города или его ID. 4 - Москва
            'catalogues': 48,  # Список разделов каталога отраслей. 48 - "Разработка, программирование"
            'no_agreement': 1  # Без вакансий, где оклад по договоренности
        }
        response = requests.get(sj_url, headers=headers, params=params)
        page += 1
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            next_page = False
            continue
        response_data = response.json()
        vacancies.extend(response_data['objects'])
        next_page = response_data['more']
    return vacancies

In [88]:
def get_sj_statistic(vacancies):
    number_of_vacancies = len(vacancies)
    if number_of_vacancies:
        salaries = [predict_rub_salary_sj(vacancy) for vacancy in vacancies]
        average_salary = int(sum(salaries)/number_of_vacancies)
        sj_statistics = {
            'vacancies_found': number_of_vacancies,
            'vacancies_processed': number_of_vacancies,
            'average_salary': average_salary
        }
        return sj_statistics


In [89]:
def make_all_language_stat_from_sj(languages):
    stat = {}
    for language in languages:
        vacancies = get_vacancies_from_sj(language)
        stat[language] = get_sj_statistic(vacancies)
    return stat

In [90]:
def make_table(site_name, statistic):
    title = '-----------------{} statistics'.format(site_name)
    if statistic:
        table_data = [[
            'lang', 'vacancies_found', 'vacancies_processed', 'average_salary'
        ]]
        for language, language_stat in statistic.items():
            if not language_stat:
                continue
            row = [language]
            for key, value in language_stat.items():
                row.append(value)
            table_data.append(row)
        return AsciiTable(table_data, title).table

In [91]:
if __name__ == '__main__':
    load_dotenv()
    languages = ['Python', 'Java', 'Javascript', 'TypeScript', 'Swift', 'Scala', 'Objective-C', 'Shell', 'Go', 'C', 
                 'PHP', 'Ruby', 'c++', 'c#', '1c']
    site_name = 'HH'
    print(make_table(site_name, make_all_language_stat_from_hh(languages)))

    site_name = 'SJ'
    print(make_table(site_name, make_all_language_stat_from_sj(languages)))

+-----------------HH statistics-+---------------------+----------------+
| lang        | vacancies_found | vacancies_processed | average_salary |
+-------------+-----------------+---------------------+----------------+
| Python      | 1980            | 562                 | 207976         |
| Java        | 1980            | 493                 | 242102         |
| Javascript  | 1980            | 939                 | 179838         |
| TypeScript  | 1720            | 499                 | 216268         |
| Swift       | 680             | 201                 | 252143         |
| Scala       | 260             | 47                  | 267638         |
| Objective-C | 240             | 67                  | 249330         |
| Shell       | 260             | 50                  | 175180         |
| Go          | 1100            | 226                 | 236125         |
| C           | 1980            | 841                 | 174731         |
| PHP         | 1960            | 893              